This notebook works as a visualization of the different functions defined in the file `ellipse.py`. It also works as small unit-tests, to check the functionality of these functions. 

### Fit ellipse to points

In [ ]:
import ellipse
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
npts = 250
tmin, tmax = np.pi / 6, 4 * np.pi / 3
x0, y0 = 4, -3.5
ap, bp = 7, 3
phi = np.pi / 4
# Get some points on the ellipse (no need to specify the eccentricity).
x, y = ellipse.get_ellipse_pts((x0, y0, ap, bp, phi), npts, tmin, tmax)
noise = 0.1
x += noise * np.random.normal(size=npts)
y += noise * np.random.normal(size=npts)

coeffs = ellipse.fit_ellipse(x, y)
print('Exact parameters:')
print('x0, y0, ap, bp, phi =', x0, y0, ap, bp, phi)
print('Fitted parameters:')
print('a, b, c, d, e, f =', coeffs)
x0, y0, ap, bp, phi = ellipse.cart_to_pol(coeffs)
print('x0, y0, ap, bp, phi = ', x0, y0, ap, bp, phi)

plt.plot(x, y, 'x')  # given points
x, y = ellipse.get_ellipse_pts((x0, y0, ap, bp, phi))
plt.plot(x, y)
plt.show()

### Project point to ellipse

This is to project the OCR recognitions to the ellipse

In [ ]:
x0, y0 = 0, 0
ap, bp = 7, 4
phi = 0
# Get some points on the ellipse (no need to specify the eccentricity).
x_e, y_e = ellipse.get_ellipse_pts((x0, y0, ap, bp, phi))
plt.plot(x_e, y_e)
plt.scatter(x0, y0)

# point to project on to the ellipse
x = 5
y = 10
point = np.array([x, y])
plt.scatter(x, y)

projected_point = ellipse.project_point(point, (x0, y0, ap, bp, phi))

x_proj, y_proj = projected_point
plt.scatter(x_proj, y_proj)
plt.scatter(x_proj, y_proj, marker = 'x')

plt.show()

### Intersect line and ellipse

This is to calculate the intersection of the needle and the ellipse.

In [ ]:
line_coeffs = np.array([1,1])
line = np.poly1d(line_coeffs)
x_start, x_end = 0.1, 2
x = np.array([x_start, x_end])

x0, y0 = 1, 1
ap, bp = 1, 2
phi = 0
# Get some points on the ellipse (no need to specify the eccentricity).
x_e, y_e = ellipse.get_ellipse_pts((x0, y0, ap, bp, phi))
plt.plot(x_e, y_e)
plt.plot(x, line(x), color='orange')

intersection_point = ellipse.get_line_ellipse_point(line_coeffs, x, (x0, y0, ap, bp, phi))
print(intersection_point)
x = intersection_point[0]
y = intersection_point[1]

plt.scatter(x, y, marker = 'o', c='red', s=100)

plt.show()

### Get middle between two points on ellipse

This is to calculate the point between the start and end point.

In [ ]:
plt.figure(figsize=(5,5))

start_point = np.array((-0.5,-2))
end_point = np.array((1.5,-1))

plt.scatter(start_point[0], start_point[1], marker = 'o', c='blue', s=100)
plt.scatter(end_point[0], end_point[1], marker = 'o', c='green', s=100)

x0, y0 = 1, 1
plt.scatter(x0, y0, marker = 'o', c='black', s=100)
ap, bp = 1, 2
phi = np.pi*5/4
ellipse_params = (x0, y0, ap, bp, phi)
# Get some points on the ellipse (no need to specify the eccentricity).
x_e, y_e = ellipse.get_ellipse_pts(ellipse_params)
plt.plot(x_e, y_e)

theta_start = ellipse.get_polar_angle(start_point, ellipse_params)
theta_end = ellipse.get_polar_angle(end_point, ellipse_params)

start_proj = ellipse.get_point_from_angle(theta_start, ellipse_params)
end_proj = ellipse.get_point_from_angle(theta_end, ellipse_params)
plt.scatter(start_proj[0], start_proj[1], marker = 'o', c='blue', s=100)
plt.scatter(end_proj[0], end_proj[1], marker = 'o', c='green', s=100)

zero_point = ellipse.get_point_from_angle(0, ellipse_params)
plt.scatter(zero_point[0], zero_point[1], marker = 'o', c='red', s=100)

theta_middle = ellipse.get_theta_middle(theta_start, theta_end)
middle = ellipse.get_point_from_angle(theta_middle, ellipse_params)

plt.scatter(middle[0], middle[1], marker = 'o', c='violet', s=100)
    

plt.show()

# Warp ellipse

Here is some code to warp the ellipse, such that it becomes a circle

In [ ]:
import cv2
import numpy as np

def warp_ellipse_to_circle(image, ellipse_center, ellipse_axes, ellipse_angle):
    image_height, image_width = image.shape[:2]

    # Define the source points (the coordinates of the four corners of the ellipse)
    x, y = ellipse_center
    major_axis, minor_axis = ellipse_axes
    source_points = np.array([[-major_axis/2, -minor_axis/2],
                            [major_axis/2, -minor_axis/2],
                            [major_axis/2, minor_axis/2],
                            [-major_axis/2, minor_axis/2]], dtype=np.float32)
    square_size = max(major_axis, minor_axis)
    destination_points = np.array([[-square_size/2, -square_size/2],
                            [square_size/2, -square_size/2],
                            [square_size/2, square_size/2],
                            [-square_size/2, square_size/2]], dtype=np.float32)


    rotation_matrix = np.array([[np.cos(phi), np.sin(phi)], [-np.sin(phi), np.cos(phi)]])
    source_points = source_points @ rotation_matrix
    destination_points = destination_points @ rotation_matrix

    source_points[:, 0] += x
    source_points[:, 1] += y
    destination_points[:, 0] += x
    destination_points[:, 1] += y

    source_points = source_points.astype(np.float32)
    destination_points = destination_points.astype(np.float32)

    # Calculate the perspective transformation matrix
    transformation_matrix = cv2.getPerspectiveTransform(source_points, destination_points)

    # Warp the image
    offsetSize=0

    warped_image = cv2.warpPerspective(image, transformation_matrix, (image_width+offsetSize, image_height+offsetSize))

    return warped_image, transformation_matrix

# Example usage
image_path = '/home/mreitsma/test_images_split/different_view_rotated/images/117_PNG_jpg.rf.fdb10fb262c1e706eba7154ed6852ed9.jpg'  # Replace with the path to your image
image = cv2.imread(image_path)

ellipse_center = (355, 208)  # Replace with the coordinates of the ellipse center
ellipse_axes = (200, 80)  # Replace with the major and minor axes lengths of the ellipse

x0 = ellipse_center[0]
y0 = ellipse_center[1]
ap = ellipse_axes[0]
bp = ellipse_axes[1]
phi = np.pi/4
x, y = ellipse.get_ellipse_pts((x0, y0, ap, bp, phi))
plt.figure()
plt.imshow(image)
plt.plot(x, y)

plt.show()




In [ ]:
warped_image, transformation_matrix = warp_ellipse_to_circle(image, ellipse_center, ellipse_axes, phi)
plt.figure()
point_warp = [440,125]
plt.imshow(warped_image)
plt.scatter(point_warp[0], point_warp[1])

This is code to remap a point in the warped image to the original image

In [ ]:
def map_point_original_image(point_warp, transformation_matrix):
    inverse_transformation_matrix = np.linalg.inv(transformation_matrix)

    point_in_original_image = cv2.perspectiveTransform(np.array([[point_warp]], dtype=np.float32),
                                                    inverse_transformation_matrix)[0][0]
    return point_in_original_image

point_in_original_image = map_point_original_image(point_warp, transformation_matrix)

plt.figure()
plt.imshow(image)
plt.scatter(point_in_original_image[0], point_in_original_image[1])


In [ ]:

image_height, image_width = image.shape[:2]

# Define the source points (the coordinates of the four corners of the ellipse)
x, y = ellipse_center
major_axis, minor_axis = ellipse_axes
source_points = np.array([[-major_axis/2, -minor_axis/2],
                            [major_axis/2, -minor_axis/2],
                            [major_axis/2, minor_axis/2],
                            [-major_axis/2, minor_axis/2]], dtype=np.float32)

rotation_matrix = np.array([[np.cos(phi), np.sin(phi)], [-np.sin(phi), np.cos(phi)]])
source_points = source_points @ rotation_matrix

source_points[:, 0] += x
source_points[:, 1] += y

ellipse_points_x, ellipse_points_y = ellipse.get_ellipse_pts((x0, y0, ap, bp, phi))
plt.figure()
plt.imshow(image)
plt.plot(ellipse_points_x, ellipse_points_y)
plt.scatter(source_points[:,0], source_points[:,1])



In [ ]:
# Define the destination points (the coordinates of the four corners of a square)
square_size = max(major_axis, minor_axis)
destination_points = np.array([[-square_size/2, -square_size/2],
                            [square_size/2, -square_size/2],
                            [square_size/2, square_size/2],
                            [-square_size/2, square_size/2]], dtype=np.float32)



destination_points = destination_points @ rotation_matrix
destination_points[:, 0] += x
destination_points[:, 1] += y

plt.figure()
plt.imshow(image)
plt.plot(ellipse_points_x, ellipse_points_y)
plt.scatter(destination_points[:,0], destination_points[:,1])

source_points = source_points.astype(np.float32)
destination_points = destination_points.astype(np.float32)


# Calculate the perspective transformation matrix
transformation_matrix = cv2.getPerspectiveTransform(source_points, destination_points)

# Warp the image
warped_image = cv2.warpPerspective(image, transformation_matrix, (image_width, image_height))